In [2]:
!pip install torch==2.4.1 transformers==4.45.1 bitsandbytes==0.44.1 peft==0.13.0 datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [21]:
import torch
from peft import PeftModel, LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm
import json
import gdown
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

In [2]:
# Define the model folder URL
model_folder_url = "https://drive.google.com/drive/folders/11qu4SrHmHoM2SBLQwoodbcd88ZziI3Vp?usp=sharing"

# Convert the folder link to a format gdown can recognize
gdown.download_folder(model_folder_url, quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 125-XJNMKToeEjchCWqNTslfGJK9Wtmtg adapter_config.json
Processing file 12Ex9G4KUqa2fE4BJv18V_mPPdPZwNmzN adapter_model.safetensors
Processing file 12GyOX02I3mtDtRQ334DW07YD1HLOxlTN README.md
Processing file 11x4tYNAYSrVwVTDvNJ_hkLaWsoWVZf9- special_tokens_map.json
Processing file 121U_sOThuoOaaBnhWcMA3yEtk_cJyaj8 tokenizer_config.json
Processing file 11vCYUzoxN7btV9zpm7YIoOk60gqAm6cL tokenizer.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=125-XJNMKToeEjchCWqNTslfGJK9Wtmtg
To: /content/epoch_1/adapter_config.json
100%|██████████| 664/664 [00:00<00:00, 1.86MB/s]
Downloading...
From: https://drive.google.com/uc?id=12Ex9G4KUqa2fE4BJv18V_mPPdPZwNmzN
To: /content/epoch_1/adapter_model.safetensors
100%|██████████| 6.40M/6.40M [00:00<00:00, 17.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=12GyOX02I3mtDtRQ334DW07YD1HLOxlTN
To: /content/epoch_1/README.md
100%|██████████| 5.11k/5.11k [00:00<00:00, 14.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=11x4tYNAYSrVwVTDvNJ_hkLaWsoWVZf9-
To: /content/epoch_1/special_tokens_map.json
100%|██████████| 636/636 [00:00<00:00, 713kB/s]
Downloading...
From: https://drive.google.com/uc?id=121U_sOThuoOaaBnhWcMA3yEtk_cJyaj8
To: /content/epoch_1/tokenizer_config.json
100%|██████████| 47.1k/47.1k [00:00<00:00, 37.0MB/s]
Do

['/content/epoch_1/adapter_config.json',
 '/content/epoch_1/adapter_model.safetensors',
 '/content/epoch_1/README.md',
 '/content/epoch_1/special_tokens_map.json',
 '/content/epoch_1/tokenizer_config.json',
 '/content/epoch_1/tokenizer.json']

In [11]:
# Google Drive file ID from your link
file_id = "18FRjHAL_N90uKv3nsKE0ChBf8fY1qFhE"
url = f"https://drive.google.com/uc?id={file_id}"

# Specify the output file path
output_path = "private_test.json"

# Download the file
gdown.download(url, output_path, quiet=False)

print(f"File downloaded and saved as '{output_path}'")

Downloading...
From: https://drive.google.com/uc?id=18FRjHAL_N90uKv3nsKE0ChBf8fY1qFhE
To: /content/private_test.json
100%|██████████| 48.4k/48.4k [00:00<00:00, 46.2MB/s]

File downloaded and saved as 'private_test.json'


In [5]:
# Dataset file URL
dataset_url = "https://drive.google.com/file/d/1mQInr4_t1IZI5ZQFsTyg-Ln9LLudFBIj"

# Download the dataset file
gdown.download(dataset_url, output="public_test.json", quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1mQInr4_t1IZI5ZQFsTyg-Ln9LLudFBIj
To: /content/private_test.json
92.4kB [00:00, 2.57MB/s]


'private_test.json'

In [12]:
checkpoint_path = "epoch_1"

In [13]:
# Load tokenizer and model from the saved checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "zake7749/gemma-2-2b-it-chinese-kyara-dpo"

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map='auto'
)

model = PeftModel.from_pretrained(base_model, checkpoint_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
def get_prompt(instruction: str) -> str:
    '''Format the instruction as a prompt for LLM.'''
    return f"請你扮演一個人工智慧國文助理，\
幫助用戶在白話文和文言文之間轉換，\
USER: {instruction} \
ASSISTANT:\
"

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
           

In [17]:
private_test_path = "private_test.json"

In [18]:
# Load dataset
with open(private_test_path, "r") as f:
    private_test_data = json.load(f)

In [23]:
# Extract instructions from the dataset for inference
instructions = [get_prompt(item["instruction"]) for item in private_test_data]

In [25]:
predictions = []

for i, instruction in enumerate(tqdm(instructions, desc="Processing Instructions")):
    # Prepare input tensor for model
    inputs = tokenizer(instruction, return_tensors="pt").to(device)

    # Generate output using the model
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=512,  # Adjust max_length if needed
            num_beams=15,     # Beam search can improve results
            early_stopping=True
        )

    # Decode the generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Extract only the part after "ASSISTANT:" in the generated output
    assistant_text = generated_text.split("ASSISTANT:")[-1].strip()

    # Append the result to predictions list
    predictions.append({
        "id": private_test_data[i]["id"],
        "output": assistant_text
    })

Processing Instructions: 100%|██████████| 250/250 [15:33<00:00,  3.73s/it]


In [26]:
# Save predictions to a JSON file
with open("prediction.json", "w") as f:
    json.dump(predictions, f, ensure_ascii=False, indent=4)

print("Inference completed and predictions saved to prediction.json.")

Inference completed and predictions saved to prediction.json.


In [27]:
base_model.to(device)

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2304, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear4bit(in_features=2304, out_features=1024, bias=False)
          (v_proj): lora.Linear4bit(
            (base_layer): Linear4bit

# Comparison

In [28]:
import random
# Select 10 random samples from the dataset
random.seed(42)  # For reproducibility
sample_data = random.sample(private_test_data, 10)

In [72]:
sample_data

[{'id': 'c795e8da-6602-441b-9632-a6eb4d5e429f',
  'instruction': '建武元年，拜為禦史中丞。\n把這句話翻譯成文言文：'},
 {'id': '0bd01970-ca46-4029-ab5a-c31b68d550d6',
  'instruction': '文言文翻譯：\n上嘉納之。'},
 {'id': '7efea98b-646a-4bd8-b85c-0118d3493506',
  'instruction': '翻譯成文言文：\n親自耕作的原因，就是錶示重視祭祀的供品。\n答案：'},
 {'id': 'bdb5da27-91b9-428d-ba77-921a53e174f4',
  'instruction': '翻譯成現代文：\n四十五年，五大夫賁攻韓，取十城。'},
 {'id': '7cfccb6e-ab89-4ae6-b38a-065ed249b12d',
  'instruction': '陳後主說： 那就把毛喜安置在一個小郡中，不許他再在朝廷參預政事。\n這句話在中國古代怎麼說：'},
 {'id': '134dca67-ff20-4582-a6aa-7610a0c00822',
  'instruction': '此一難忍也。\n翻譯成白話文：'},
 {'id': '1c494ae3-1df0-4ce8-ae4f-3857864dec25',
  'instruction': '到崇政殿受冊就夠瞭！ 元宵節燈宴，太後的母親應當入宮，太後製止說 ：夫人登樓，皇上一定增加禮儀，這樣由於我的原因而超越典章製度，於心很不安啊！ 隻令賜給她燈燭，於是每年以此為常例。\n這句話在古代怎麼說：'},
 {'id': '2c0caaf9-904a-425f-b516-de4dd438d66c',
  'instruction': '以己之心為根據來要求彆人，把自己的好惡也用到彆人身上。\n幫我把這句話翻譯成文言文'},
 {'id': '364fee8a-45bc-4f58-92d9-cafc8085fe09',
  'instruction': '人或鑿崖取之，即風雷為變。\n翻譯成白話文：'},
 {'id': '91650fd4-6b91-474a-b301-9e0187d4f9a3',

## LORA

In [60]:
# Extract instructions from the dataset for inference
instructions = [get_prompt(item["instruction"]) for item in sample_data]

In [61]:
lora_predictions = []

for i, instruction in enumerate(tqdm(instructions, desc="Processing Instructions")):
    # Prepare input tensor for model
    inputs = tokenizer(instruction, return_tensors="pt").to(device)

    # Generate output using the model
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=512,  # Adjust max_length if needed
            num_beams=15,     # Beam search can improve results
            early_stopping=True
        )

    # Decode the generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    # Extract only the part after "ASSISTANT:" in the generated output
    assistant_text = generated_text.split("ASSISTANT:")[-1].strip()

    # Append the result to predictions list
    lora_predictions.append({
        "id": private_test_data[i]["id"],
        "output": assistant_text
    })

Processing Instructions: 100%|██████████| 10/10 [00:26<00:00,  2.66s/it]


In [62]:
# Save predictions to a JSON file
with open("lora_prediction.json", "w") as f:
    json.dump(lora_predictions, f, ensure_ascii=False, indent=4)

print("Inference completed and predictions saved to lora_prediction.json.")

Inference completed and predictions saved to lora_prediction.json.


## Zero shot

In [54]:
def zero_shot_prompt(instruction: str) -> str:
    return f"\
請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換\
你被給予的指示中會有明確要求要轉換成文言文還是白話文\
如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案\
如果這個要求在句尾，請直接回答：文言文答案/白話文答案\
以下是你被給予的指示：\
USER:{instruction} \
ASSISTANT:\
"

In [55]:
print(zero_shot_prompt("djd"))

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:djd ASSISTANT:


In [56]:
zero_shot_instructions = [zero_shot_prompt(item["instruction"]) for item in sample_data]

In [57]:
zero_shot_instructions[0]

'請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:建武元年，拜為禦史中丞。\n把這句話翻譯成文言文： ASSISTANT:'

In [58]:
zero_shot_predictions = []

for i, instruction in enumerate(tqdm(zero_shot_instructions, desc="Processing Instructions")):
    # Prepare input tensor for model
    inputs = tokenizer(instruction, return_tensors="pt").to(device)

    # Generate output using the model
    with torch.no_grad():
        generated_ids = base_model.generate(
            **inputs,
            max_length=512,  # Adjust max_length if needed
            num_beams=15,     # Beam search can improve results
            early_stopping=True
        )

    # Decode the generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    print(generated_text)

    # Extract only the part after "ASSISTANT:" in the generated output
    assistant_text = generated_text.split("ASSISTANT:")[-1].strip()
    print(assistant_text)
    # Append the result to predictions list
    zero_shot_predictions.append({
        "id": private_test_data[i]["id"],
        "output": assistant_text
    })

Processing Instructions:  10%|█         | 1/10 [00:01<00:14,  1.61s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:建武元年，拜為禦史中丞。
把這句話翻譯成文言文： ASSISTANT:建武元年，拜禦史中丞。
建武元年，拜禦史中丞。


Processing Instructions:  20%|██        | 2/10 [00:02<00:10,  1.27s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:文言文翻譯：
上嘉納之。 ASSISTANT:答案：上嘉之。
答案：上嘉之。


Processing Instructions:  30%|███       | 3/10 [00:04<00:10,  1.55s/it]

...

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:以己之心為根據來要求彆人，把自己的好惡也用到彆人身上。
幫我把這句話翻譯成文言文 ASSISTANT:以己之心為根據，要求彆人，把自己的好惡也用於彆人。
以己之心為根據，要求彆人，把自己的好惡也用於彆人。


Processing Instructions:  90%|█████████ | 9/10 [00:26<00:03,  3.47s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:人或鑿崖取之，即風雷為變。
翻譯成白話文： ASSISTANT:人或鑿崖取之，即風雷為變。
人或鑿崖取之，即風雷為變。


Processing Instructions: 100%|██████████| 10/10 [00:27<00:00,  2.72s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是你被給予的指示：USER:翻譯成文言文：
她高壽多少？
答案： ASSISTANT:高壽多少？
高壽多少？


In [59]:
# Save predictions to a JSON file
with open("zero_shot_prediction.json", "w") as f:
    json.dump(zero_shot_predictions, f, ensure_ascii=False, indent=4)

print("Inference completed and predictions saved to zero_shot_prediction.json.")

Inference completed and predictions saved to zero_shot_prediction.json.


## Few-shot

In [66]:
def few_shot_prompt(instruction: str) -> str:
    return f"\
請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換\
你被給予的指示中會有明確要求要轉換成文言文還是白話文\
如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案\
如果這個要求在句尾，請直接回答：文言文答案/白話文答案\
以下是幾個範例：\
範例一：\
USER:翻譯成文言文：\n於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。\
ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。\
範例二：\
USER:文言文翻譯：\n靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 \
ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。\
範例三：\
USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。\n這句話在古代怎麼說： \
ASSISTANT:以後幸長官留意，勿令如此。\
現在是你被給予的指示：\
USER:{instruction} \
ASSISTANT:\
"

In [67]:
few_shot_instructions = [few_shot_prompt(item["instruction"]) for item in sample_data]

In [68]:
few_shot_instructions[0]

'請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是幾個範例：範例一：USER:翻譯成文言文：\n於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。範例二：USER:文言文翻譯：\n靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。範例三：USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。\n這句話在古代怎麼說： ASSISTANT:以後幸長官留意，勿令如此。現在是你被給予的指示：USER:建武元年，拜為禦史中丞。\n把這句話翻譯成文言文： ASSISTANT:'

In [69]:
few_shot_predictions = []

for i, instruction in enumerate(tqdm(few_shot_instructions, desc="Processing Instructions")):
    # Prepare input tensor for model
    inputs = tokenizer(instruction, return_tensors="pt").to(device)

    # Generate output using the model
    with torch.no_grad():
        generated_ids = base_model.generate(
            **inputs,
            max_length=512,  # Adjust max_length if needed
            num_beams=15,     # Beam search can improve results
            early_stopping=True
        )

    # Decode the generated text
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(generated_text)
    # Extract only the part after "ASSISTANT:" in the generated output
    assistant_text = generated_text.split("ASSISTANT:")[-1].strip()
    print(assistant_text)

    # Append the result to predictions list
    few_shot_predictions.append({
        "id": private_test_data[i]["id"],
        "output": assistant_text
    })

Processing Instructions:  10%|█         | 1/10 [00:01<00:15,  1.75s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是幾個範例：範例一：USER:翻譯成文言文：
於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。範例二：USER:文言文翻譯：
靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。範例三：USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。
這句話在古代怎麼說： ASSISTANT:以後幸長官留意，勿令如此。現在是你被給予的指示：USER:建武元年，拜為禦史中丞。
把這句話翻譯成文言文： ASSISTANT:建武元年，拜禦史中丞。
建武元年，拜禦史中丞。


Processing Instructions:  20%|██        | 2/10 [00:03<00:11,  1.48s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是幾個範例：範例一：USER:翻譯成文言文：
於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。範例二：USER:文言文翻譯：
靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。範例三：USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。
這句話在古代怎麼說： ASSISTANT:以後幸長官留意，勿令如此。現在是你被給予的指示：USER:文言文翻譯：
上嘉納之。 ASSISTANT:答案：上嘉納之。
答案：上嘉納之。


Processing Instructions:  30%|███       | 3/10 [00:05<00:13,  1.87s/it]

...

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是幾個範例：範例一：USER:翻譯成文言文：
於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。範例二：USER:文言文翻譯：
靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。範例三：USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。
這句話在古代怎麼說： ASSISTANT:以後幸長官留意，勿令如此。現在是你被給予的指示：USER:以己之心為根據來要求彆人，把自己的好惡也用到彆人身上。
幫我把這句話翻譯成文言文 ASSISTANT:以己心為之，以己惡為之。
以己心為之，以己惡為之。


Processing Instructions:  90%|█████████ | 9/10 [00:28<00:03,  3.50s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是幾個範例：範例一：USER:翻譯成文言文：
於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。範例二：USER:文言文翻譯：
靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。範例三：USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。
這句話在古代怎麼說： ASSISTANT:以後幸長官留意，勿令如此。現在是你被給予的指示：USER:人或鑿崖取之，即風雷為變。
翻譯成白話文： ASSISTANT:人或鑿崖取之，即風雷為變。
人或鑿崖取之，即風雷為變。


Processing Instructions: 100%|██████████| 10/10 [00:29<00:00,  2.95s/it]

請你扮演一個人工智慧國文助理，幫助用戶在白話文和文言文之間轉換你被給予的指示中會有明確要求要轉換成文言文還是白話文如果這個要求在句首，請用這種格式回答：答案：文言文答案/白話文答案如果這個要求在句尾，請直接回答：文言文答案/白話文答案以下是幾個範例：範例一：USER:翻譯成文言文：
於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。ASSISTANT:帝乃使慶之從父兄子直閣將軍攸之賜慶之藥。範例二：USER:文言文翻譯：
靈鑒忽臨，忻歡交集，乃迴燈拂席以延之。 ASSISTANT:答案：靈仙忽然光臨，趙旭歡欣交集，於是他就把燈點亮，拂拭乾淨床席來延請仙女。範例三：USER:希望您以後留意，不要再齣這樣的事，你的小女兒病就會好。
這句話在古代怎麼說： ASSISTANT:以後幸長官留意，勿令如此。現在是你被給予的指示：USER:翻譯成文言文：
她高壽多少？
答案： ASSISTANT:高壽多少？
高壽多少？


In [70]:
# Save predictions to a JSON file
with open("few_shot_prediction.json", "w") as f:
    json.dump(few_shot_predictions, f, ensure_ascii=False, indent=4)

print("Inference completed and predictions saved to few_shot_prediction.json.")

Inference completed and predictions saved to few_shot_prediction.json.
